# Практическая работа 3
## Тема: Объединение, связывание и изменение формы данных.
## Цель работы: 
Знакомство с технологией объединения, связывания и изменения формы данных.

### Настройка зависимостей

In [291]:
import pandas as pd

### Задание 1. Данные по штатам США
На основе наборов данных по штатам США и их населению отсортировать штаты и территорию США по плотности населения в 2010 году.

1. Создайте для каждого набора данных [state-abbrevs.csv](./state-abbrevs.csv), [state-areas.csv](./state-areas.csv) и [state-population.csv](./state-population.csv) отдельный датафрейм.

In [292]:
state_abbrevs = pd.read_csv("state-abbrevs.csv")
state_areas = pd.read_csv("state-areas.csv")
state_population = pd.read_csv("state-population.csv")

state_areas.rename(columns={"area (sq. mi)": "area"}, inplace=True)

2. С использованием метода `merge` выполните операцию объединения для получения полного имени штата в объекте DataFrame для населения

In [293]:
state_population = state_population.merge(
    state_abbrevs, left_on="state/region", right_on="abbreviation", how="outer"
).drop(columns=["abbreviation"])

state_population

,state/region,ages,year,population,state
0,AK,total,1990,553290.0,Alaska
1,AK,under18,1990,177502.0,Alaska
2,AK,total,1992,588736.0,Alaska
3,AK,under18,1991,182180.0,Alaska
4,AK,under18,1992,184878.0,Alaska
...,...,...,...,...,...
2539,WY,under18,1993,137458.0,Wyoming
2540,WY,total,1991,459260.0,Wyoming
2541,WY,under18,1991,136720.0,Wyoming
2542,WY,under18,1990,136078.0,Wyoming


3. Проверьте, не было ли каких-то несовпадений (пустых значений).

In [294]:
state_population.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

4. Подумайте, с чем может быть связано появление пустых значений? Попробуйте это исправить.

- Причина наличия пустых значений

    Посмотрим на следующий DataFrame:

In [295]:
state_population[state_population.isnull().any(axis=1)]

,state/region,ages,year,population,state
1872,PR,under18,1990,NaN,NaN
1873,PR,total,1990,NaN,NaN
1874,PR,total,1991,NaN,NaN
1875,PR,under18,1991,NaN,NaN
1876,PR,total,1993,NaN,NaN
...,...,...,...,...,...
2203,USA,total,2010,309326295.0,NaN
2204,USA,under18,2011,73902222.0,NaN
2205,USA,total,2011,311582564.0,NaN
2206,USA,under18,2012,73708179.0,NaN


В данном DataFrame есть две причины появления пустых значений:

1. В переменной **state/region** есть ячейки со значением "PR". Это аббревиатура Пуэрто-Рико. Это территория не является полноценным штатом США, поэтому в Dataframe `state_abbrevs` её нет (переменная **state** незаполненная). Также есть пропуски в переменной **population**

2. В переменной **state/region** есть ячейки со значением "USA". Это аббревиатура Соединненых Штатов Америки. Это не штат, а государство, включающее все штаты, поэтому в Dataframe `state_abbrevs` её нет (переменная **state** незаполненная)

- Как исправить?

1. Изменение переменных **population** и **state** для наблюдений о Пуэрто-Рико.

    - Для **population** установим среднее значение текущего года по всем штатам, разделяя по возрастам (переменной **ages**)
    - Для **state** - "Puerto Rico"

In [296]:
is_pr = state_population["state/region"] == "PR"

not_pr = state_population[~is_pr]
avg_population = not_pr.groupby(["ages", "year"])["population"].mean().round(2)

state_population.loc[is_pr, "population"] = state_population[is_pr].apply(
    lambda row: avg_population[row["ages"], row["year"]], axis=1
)
state_population.loc[is_pr, "state"] = "Puerto Rico"

state_population[is_pr].head()

,state/region,ages,year,population,state
1872,PR,under18,1990,2469942.77,Puerto Rico
1873,PR,total,1990,9600877.46,Puerto Rico
1874,PR,total,1991,9730036.23,Puerto Rico
1875,PR,under18,1991,2512039.15,Puerto Rico
1876,PR,total,1993,9996869.04,Puerto Rico


2. Удалим наблюдения со значением переменной **state/region**, равным "USA"

In [297]:
not_usa = state_population["state/region"] != "USA"

state_population = state_population[not_usa]

5. Из получившегося датафрейма выберите часть данных, соответствующих 2010 году и всему населению.

In [298]:
mask = (state_population.year == 2010) & (state_population.ages == "total")

total_2010_population = state_population[mask]

total_2010_population.head()

,state/region,ages,year,population,state
43,AK,total,2010,713868.0,Alaska
51,AL,total,2010,4785570.0,Alabama
141,AR,total,2010,2922280.0,Arkansas
149,AZ,total,2010,6408790.0,Arizona
197,CA,total,2010,37333601.0,California


5.1 Вычислите плотность населения.

In [299]:
total_2010_population = total_2010_population.merge(state_areas)

total_2010_population["density"] = (
    total_2010_population.population / total_2010_population.area
).round(2)

total_2010_population.head()

,state/region,ages,year,population,state,area,density
0,AK,total,2010,713868.0,Alaska,656425,1.09
1,AL,total,2010,4785570.0,Alabama,52423,91.29
2,AR,total,2010,2922280.0,Arkansas,53182,54.95
3,AZ,total,2010,6408790.0,Arizona,114006,56.21
4,CA,total,2010,37333601.0,California,163707,228.05


5.2 Выполните сортировку штатов по убыванию плотности населения. Какой штат получился самым густонаселенным, а какой самым малонаселенным?

In [300]:
sorted_total_2010 = total_2010_population.sort_values(by="density", ascending=False)

sorted_total_2010.head()

,state/region,ages,year,population,state,area,density
7,DC,total,2010,605125.00,District of Columbia,68,8898.90
39,PR,total,2010,11897165.19,Puerto Rico,3515,3384.68
31,NJ,total,2010,8802707.00,New Jersey,8722,1009.25
40,RI,total,2010,1052669.00,Rhode Island,1545,681.34
6,CT,total,2010,3579210.00,Connecticut,5544,645.60


In [301]:
state_max_density = sorted_total_2010.iloc[0]
state_min_density = sorted_total_2010.iloc[-1]

print(
    f"Самый густонаселенный штат: {state_max_density.state} "
    + f"({state_max_density["state/region"]})"
)
print(
    f"Самый малонаселенный штат: {state_min_density.state} "
    + f"({state_min_density["state/region"]})"
)

Самый густонаселенный штат: District of Columbia (DC)
Самый малонаселенный штат: Alaska (AK)


### Задание 2. Данные о продажах
Набор данных [orders.csv](./orders.csv) содержит информацию о позициях в заказе, набор данных [Products.csv](./Products.csv) - выгрузка из справочника товаров.

Ваша задача:

0. Импортировать данные

In [302]:
orders = pd.read_csv("orders.csv", sep=";")
products = pd.read_csv("Products.csv", sep=";")

1. Объединить наборы данных и вывести информацию о заказах в следующем формате:

In [303]:
orders = orders.merge(products, left_on="ID товара", right_on="Product_ID", 
                      how="left")

orders.head()

,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество,Product_ID,Name,Price,CURRENCY
0,09.11.2019 21:55:51,9,10,"Принят, ожидается оплата",Нет,Нет,Нет,103,5,103.0,"Носки Подарочные, муж",199.0,RUR
1,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,86,100,86.0,"Носки Простые, муж",45.0,RUR
2,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,104,10,104.0,"Носки Подарочные, жен",249.0,RUR
3,09.11.2019 12:50:07,7,8,"Принят, ожидается оплата",Нет,Нет,Нет,104,7,104.0,"Носки Подарочные, жен",249.0,RUR
4,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,104,5,104.0,"Носки Подарочные, жен",249.0,RUR


2. Найдите и удалите некорректные данные.

In [304]:
orders.dropna(inplace=True)

3. Для каждого заказа выполните расчёт стоимости с учётом кол-ва позиций.

In [305]:
orders["Стоимость"] = orders["Price"] * orders["Количество"]

orders.groupby("Order ID")["Стоимость"].sum()

Order ID
1    13043.0
2    17096.0
3     1344.0
4       50.0
5      999.0
6     2240.0
7     1743.0
8     6990.0
9      995.0
Name: Стоимость, dtype: float64

4. Выведите все заказы ожидающие оплату.

In [306]:
orders[orders["Статус"] == "Принят, ожидается оплата"]

,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество,Product_ID,Name,Price,CURRENCY,Стоимость
0,09.11.2019 21:55:51,9,10,"Принят, ожидается оплата",Нет,Нет,Нет,103,5,103.0,"Носки Подарочные, муж",199.0,RUR,995.0
1,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,86,100,86.0,"Носки Простые, муж",45.0,RUR,4500.0
2,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,104,10,104.0,"Носки Подарочные, жен",249.0,RUR,2490.0
3,09.11.2019 12:50:07,7,8,"Принят, ожидается оплата",Нет,Нет,Нет,104,7,104.0,"Носки Подарочные, жен",249.0,RUR,1743.0
4,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,104,5,104.0,"Носки Подарочные, жен",249.0,RUR,1245.0
5,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,103,5,103.0,"Носки Подарочные, муж",199.0,RUR,995.0
7,08.11.2019 08:36:22,4,9,"Принят, ожидается оплата",Нет,Нет,Да,91,1,91.0,"Носки Честные, муж",50.0,RUR,50.0


5. На какую сумму купили и оплатили носков любых видов (как мужских, так и женских)?

In [307]:
is_paid_socks = (orders["Оплачен"] == "Да") & orders["Name"].str.contains(
    "носки", case=False, na=False
)

orders[is_paid_socks]["Стоимость"].sum()

np.float64(1389.0)

#### Экспорт в html

In [308]:
from os import system

system("jupyter nbconvert --to html practice_3.ipynb")

[NbConvertApp] Converting notebook practice_3.ipynb to html
[NbConvertApp] Writing 324547 bytes to practice_3.html


0